# Neo4j and Cypher Basics with Python: Social Network Analysis

In this notebook, we will practice Neo4j, a graph database, to model a simple social network. We will cover the concepts of:
- Connecting to a Neo4j database from Python
- Creating nodes and relationships
- Running Cypher queries

**Create a new project**:
A project in Neo4j is a container for one or more databases. It helps to organize multiple databases that are related to a specific task, theme, or application. For example, we might have a project called "Social Network" that contains databases for development, testing, and production environments.
   - Open Neo4j Desktop.
   - Click on "New" under the "Projects" section.
   - Name the project (e.g., "Social Network").
   
**Create a new database**:
A database within a project is an actual instance where the graph data is stored. Each database has its own data and schema. For example, within the "Social Network" project, we might have a database named "SocialNetworkDB" where we store all user nodes and their relationships.
   - Click on "Add" and choose "Local DBMS".
   - Name your database (e.g., "Social Network DBMS"), set a password (e.g., "SocialNetworkDBMS"), and click "Create".
   - Click "Start" to start your new database.


#### Setting up the connection to Neo4j from Python
First, we need to set up the connection to the Neo4j database. Make sure the Neo4j database is running.

In [1]:
from neo4j import GraphDatabase

# Define the connection details
uri = "bolt://localhost:7687"  # default URI for Neo4j
user = "neo4j"  # default user
password = "SocialNetworkDBMS"

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(user, password))

# Verify the connection
def test_connection(driver):
    try:
        with driver.session() as session:
            result = session.run("RETURN 'Connection successful' AS message")
            for record in result:
                print(record["message"])
    except Exception as e:
        print(f"Connection failed: {e}")

test_connection(driver)

Connection successful


### Creating nodes
Nodes are the entities in a graph. In our social network, users will be nodes.

In [2]:
# Function to create a user node
def create_user(tx, name, age):
    tx.run("CREATE (u:User {name: $name, age: $age})", name=name, age=age)

# Add some users to the database
with driver.session() as session:
    session.execute_write(create_user, "Alice", 30)
    session.execute_write(create_user, "Bob", 25)
    session.execute_write(create_user, "Charlie", 35)
    session.execute_write(create_user, "Diana", 28)
    session.execute_write(create_user, "Eli", 33)
    session.execute_write(create_user, "Felix", 23)

print("Users created successfully.")

Users created successfully.


##### Explanation of the CREATE query

The `CREATE` query is used to create nodes and relationships in the graph. Here's the structure:

```cypher
CREATE (alias:Label {property1: value1, property2: value2, ...})
```

* `alias`: A variable that refers to the node. Variables in Cypher are used to refer to nodes, relationships, and paths in queries.
* `Label`: The label that categorizes the node.
* `property1, property2, ...`: Properties of the node.

In our example:

```cypher
CREATE (u:User {name: $name, age: $age})
```

* `(u:User ...)` creates a node with the label `User`. Here, `u` is a variable representing a node with the label User.
* `{name: $name, age: $age}` sets the properties `name` and `age`.

### Creating relationships

#### Creating directed relationships
Relationships connect nodes. Relationships can be directed, indicating a one-way connection between nodes. For example, a "follower" relationship where one user follows another.

In [3]:
# Function to create a FOLLOW relationship (directed)
def create_follower(tx, follower, followed):
    tx.run("""
    MATCH (a:User {name: $follower}), (b:User {name: $followed})
    CREATE (a)-[:FOLLOW]->(b)
    """, follower=follower, followed=followed)

# Add some follower relationships to the database
with driver.session() as session:
    session.execute_write(create_follower, "Alice", "Charlie")
    session.execute_write(create_follower, "Alice", "Felix")
    session.execute_write(create_follower, "Bob", "Eli")
    session.execute_write(create_follower, "Eli", "Bob")

print("Follower relationships created successfully.")

Follower relationships created successfully.


##### Explanation of the MATCH and CREATE query

The `MATCH` query is used to find existing nodes, and the `CREATE` query is used to create relationships. Here's the structure:

```cypher
MATCH (alias1:Label {property1: value1}), (alias2:Label {property2: value2})
CREATE (alias1)-[:RELATIONSHIP_TYPE]->(alias2)
```

* `MATCH (alias1:Label {property1: value1})`: Finds nodes that match the given label and property.
* `CREATE (alias1)-[:RELATIONSHIP_TYPE]->(alias2)`: Creates a relationship of type `RELATIONSHIP_TYPE` between the matched nodes found in the `MATCH` query. The colon indicates that what follows is the type of relationship.

In our example:

```cypher
MATCH (a:User {name: $follower}), (b:User {name: $followed})
CREATE (a)-[:FOLLOW]->(b)
```

* `MATCH (a:User {name: $follower}), (b:User {name: $followed})`: Finds users named `$follower` and `$followed`.
* `CREATE (a)-[:FOLLOW]->(b)`: Creates a `FOLLOW` relationship from user `a` to user `b`.

#### Creating bidirectional relationships
To model a bidirectional relationship, such as friendship, we need to create two directed relationships in opposite directions. In our social network, users can be friends with each other.

In [4]:
# Function to create a FRIEND relationship between two users
def create_friendship(tx, name1, name2):
    tx.run("""
    MATCH (a:User {name: $name1}), (b:User {name: $name2})
    CREATE (a)-[:FRIEND]->(b), (b)-[:FRIEND]->(a)
    """, name1=name1, name2=name2)

# Add some friendships to the database
with driver.session() as session:
    session.execute_write(create_friendship, "Alice", "Bob")
    session.execute_write(create_friendship, "Alice", "Charlie")
    session.execute_write(create_friendship, "Bob", "Diana")
    session.execute_write(create_friendship, "Diana", "Eli")
    session.execute_write(create_friendship, "Bob", "Felix")
    session.execute_write(create_friendship, "Alice", "Felix")

print("Friendships created successfully.")

Friendships created successfully.


##### Explanation CREATE query for bidirectional relationships

In our example:

```cypher
MATCH (a:User {name: $name1}), (b:User {name: $name2})
CREATE (a)-[:FRIEND]->(b), (b)-[:FRIEND]->(a)
```

* `MATCH (a:User {name: $name1}), (b:User {name: $name2})`: Finds users named `$name1` and `$name2`.
* `CREATE (a)-[:FRIEND]->(b), (b)-[:FRIEND]->(a)`: Creates a `FRIEND` relationship for users `a` and user `b`.
    * `(a)-[:FRIEND]->(b)`: Creates a `FRIEND` relationship from user `a` to user `b`.
    * `(b)-[:FRIEND]->(a)`: Creates a `FRIEND` relationship from user `b` to user `a`.

### Adding different types of nodes
We can have different types of nodes representing various entities in the domain model. For example, in a social network, besides User nodes, we might have Post and Comment nodes.

In [5]:
# Function to create a post node
def create_post(tx, id, content, timestamp, latitude, longitude):
    tx.run("""
    CREATE (p:Post {
        id: $id,
        content: $content,
        timestamp: datetime($timestamp),  // Handle timestamp as a datetime value
        location: point({latitude: $latitude, longitude: $longitude})  // Handle spatial location
    })
    """, id=id, content=content, timestamp=timestamp, latitude=latitude, longitude=longitude)

# Function to create a comment node with spatial handling
def create_comment(tx, id, content, timestamp, latitude, longitude):
    tx.run("""
    CREATE (c:Comment {
        id: $id,
        content: $content,
        timestamp: datetime($timestamp),  // Handle timestamp as a datetime value
        location: point({latitude: $latitude, longitude: $longitude})  // Handle spatial location
    })
    """, id=id, content=content, timestamp=timestamp, latitude=latitude, longitude=longitude)

# Add some posts and comments to the database
with driver.session() as session:
    session.execute_write(create_post, 1, "Hello, world!", "2024-07-16T10:00:00Z", 40.7128, -74.0060)
    session.execute_write(create_post, 2, "Good night!", "2024-07-16T11:00:00Z", 34.0522, -118.2437)
    session.execute_write(create_comment, 1, "Great post!", "2024-07-16T12:00:00Z", 51.5074, -0.1278)
    session.execute_write(create_comment, 2, "Very informative.", "2024-07-16T12:30:00Z", -33.8688, 151.2093)

print("Posts and comments created successfully.")

Posts and comments created successfully.


In these queries, we create two types of nodes, `Post` and `Comment`, each with properties such as `id`, `content`, `timestamp` and `location`.

##### Handling time and spatial values

* `datetime($timestamp)`: This function converts the timestamp string into a datetime type. Neo4j can then store and process this as a temporal value.
* `point({latitude: $latitude, longitude: $longitude})`: This function creates a spatial point using latitude and longitude. Neo4j stores this as a spatial point data type, enabling geographical queries.

### Creating relationships between Users, Posts, and Comments
Let's add the relationships between users and posts (e.g., POSTED), and between users and comments (e.g., COMMENTED).

In [6]:
# Function to create a POSTED relationship between a user and a post
def create_posted_relationship(tx, user_name, post_id):
    tx.run("""
    MATCH (u:User {name: $user_name}), (p:Post {id: $post_id})
    CREATE (u)-[:POSTED]->(p)
    """, user_name=user_name, post_id=post_id)

# Function to create a COMMENTED relationship between a user and a comment
def create_commented_relationship(tx, user_name, comment_id):
    tx.run("""
    MATCH (u:User {name: $user_name}), (c:Comment {id: $comment_id})
    CREATE (u)-[:COMMENTED]->(c)
    """, user_name=user_name, comment_id=comment_id)
    
    # Function to create a HAS_COMMENT relationship between a post and a comment
def create_has_comment_relationship(tx, post_id, comment_id):
    tx.run("""
    MATCH (p:Post {id: $post_id}), (c:Comment {id: $comment_id})
    CREATE (p)-[:HAS_COMMENT]->(c)
    """, post_id=post_id, comment_id=comment_id)
    
# Add relationships to the database
with driver.session() as session:
    # Alice and Bob post something
    session.execute_write(create_posted_relationship, "Alice", 1)
    session.execute_write(create_posted_relationship, "Bob", 2)
    
    # Alice and Charlie comment on posts
    session.execute_write(create_commented_relationship, "Alice", 1)
    session.execute_write(create_commented_relationship, "Charlie", 2)
    
    # Connect comments to posts
    session.execute_write(create_has_comment_relationship, 1, 2)
    session.execute_write(create_has_comment_relationship, 2, 1)

print("Relationships created successfully.")

Relationships created successfully.


### Adding properties to relationships
Relationships can have properties just like nodes. These properties can store various types of data such as strings, numbers, booleans, and arrays. This is useful for adding metadata to the relationships.

In [7]:
# Function to create a LIKES relationship with properties
def create_like(tx, user, post_id, timestamp):
    tx.run("""
    MATCH (u:User {name: $user}), (p:Post {id: $post_id})
    CREATE (u)-[:LIKES {timestamp: $timestamp}]->(p)
    """, user=user, post_id=post_id, timestamp=timestamp)

# Add some likes with properties to the database
with driver.session() as session:
    session.execute_write(create_like, "Alice", 1, "2024-07-16T12:00:00Z")
    session.execute_write(create_like, "Bob", 2, "2024-07-16T12:30:00Z")

print("Likes with properties created successfully.")

Likes with properties created successfully.


In these queries, we create LIKES relationships between users and posts with an additional timestamp property. A property of the LIKES relationship indicating when the like was made. This property is stored as a string representing the date and time.

* `MATCH (u:User {name: $user}), (p:Post {id: $post_id})`: Finds the user and post nodes based on the specified properties.
* `CREATE (u)-[:LIKES {timestamp: $timestamp}]->(p)`: Creates a LIKES relationship between the user and post with a timestamp property.

### Deleting nodes and relationships

We can also delete nodes and relationships using the `DELETE` query.

In [8]:
# Function to delete a user node
def delete_user(tx, name):
    tx.run("MATCH (u:User {name: $name}) DETACH DELETE u", name=name)

# Function to delete a FOLLOW relationship
def delete_follow(tx, follower_name, followee_name):
    tx.run("""
    MATCH (a:User {name: $follower_name})-[r:FOLLOW]->(b:User {name: $followee_name})
    DELETE r
    """, follower_name=follower_name, followee_name=followee_name)

# Function to delete a FRIEND relationship
def delete_friendship(tx, name1, name2):
    tx.run("""
    MATCH (a:User {name: $name1})-[r:FRIEND]->(b:User {name: $name2})
    DELETE r
    """, name1=name1, name2=name2)
    tx.run("""
    MATCH (b:User {name: $name2})-[r:FRIEND]->(a:User {name: $name1})
    DELETE r
    """, name1=name1, name2=name2)

# Delete user 'Diana'
with driver.session() as session:
    session.execute_write(delete_user, "Diana")

print("User Diana deleted successfully.")

# Delete follow relationship between 'Alice' and 'Felix'
with driver.session() as session:
    session.execute_write(delete_follow, "Alice", "Felix")

print("Follow relationship between Alice and Felix deleted successfully.")

# Delete friendship between 'Alice' and 'Felix'
with driver.session() as session:
    session.execute_write(delete_friendship, "Felix", "Alice")

print("Friendship between Felix and Alice deleted successfully.")

User Diana deleted successfully.
Follow relationship between Alice and Felix deleted successfully.
Friendship between Felix and Alice deleted successfully.


##### Explanation of the DELETE query

The `DELETE` query is used to remove nodes and relationships from the graph.

* Deleting a node without relationships - When we delete a node, we are removing that node and all the data associated with it from the graph. If the node does not have any relationships, it can be deleted directly using:
    ```cypher
    MATCH (u:User {name: $name})
    DELETE u
    ```

* Deleting a node with relationships: 
If the node has relationships, we must also handle those relationships. By default, Neo4j will not allow us to delete a node that still has relationships because it would leave dangling references in the graph. To delete a node and its relationships, you can use the `DETACH DELETE` command. This command removes the node and all relationships connected to it:

    ```cypher
    MATCH (u:User {name: $name})
    DETACH DELETE u
    ```

    - `DETACH DELETE u`: Ensures that the user and all relationships connected to the user are deleted. This prevents leaving orphaned relationships in the database.

* Deleting relationships only: 
We can delete a relationship between nodes without deleting the nodes by matching the relationship and then using the DELETE command:

    ```cypher
    MATCH (a:User {name: $name1})-[r:FRIEND]->(b:User {name: $name2})
    DELETE r
    ```
    
    - `MATCH (a:User {name: $name1})-[r:FRIEND]->(b:User {name: $name2})`: Finds the `FRIEND` relationship between `$name1` and `$name2`. The `r` is an alias used for the relationship in the query. It allows us to refer to and manipulate that specific relationship.
    - `DELETE r`: Deletes the matched relationship.
    
* Deleting bi-directional relationships: 
To delete a `FRIEND` relationship from both sides, we need to run two DELETE commands:

    ```
    MATCH (a:User {name: $name1})-[r:FRIEND]->(b:User {name: $name2})
    DELETE r
    MATCH (b:User {name: $name2})-[r:FRIEND]->(a:User {name: $name1})
    DELETE r
    ```
    
    - This ensures that the `FRIEND` relationship is deleted in both directions.   
    

### Using `Merge` clause to add nodes and relationships

The `MERGE` clause is used to either match existing nodes and relationships or create new ones if they do not exist. It is useful for ensuring that a certain pattern is present in the database, whether by finding existing elements or creating new ones as necessary.

In [9]:
# Function to create or match a user node
def create_or_match_user(tx, name, age):
    tx.run("""
    MERGE (u:User {name: $name})
    SET u.age = $age
    """, name=name, age=age)

# Add or match some users in the database
with driver.session() as session:
    session.execute_write(create_or_match_user, "Bob", 25)  # Already exist
    session.execute_write(create_or_match_user, "Gabriella", 31) # New node

print("Users created or matched successfully.")


################# Using MERGE for both nodes and relationship  #################

# Function to create or match a FOLLOW relationship (directed)
def create_or_match_follower(tx, follower, followed):
    tx.run("""
    MERGE (a:User {name: $follower})
    MERGE (b:User {name: $followed})
    MERGE (a)-[:FOLLOW]->(b)
    """, follower=follower, followed=followed)
    
# Add some follower relationships to the database
with driver.session() as session:
    session.execute_write(create_or_match_follower, "Alice", "Charlie")  # Nodes and relationship already exist
    session.execute_write(create_or_match_follower, "Hannah", "Felix")  # New relationship and new node

print("Follower relationships created or matched successfully.")


# Function to create or match a FRIEND relationship between two users
def create_or_match_friendship(tx, name1, name2):
    tx.run("""
    MERGE (a:User {name: $name1})
    MERGE (b:User {name: $name2})
    MERGE (a)-[:FRIEND]->(b)
    MERGE (b)-[:FRIEND]->(a)
    """, name1=name1, name2=name2)
    
# Add some friendships to the database
with driver.session() as session:
    session.execute_write(create_or_match_friendship, "Alice", "Charlie")  # Nodes and relationship already exist
    session.execute_write(create_or_match_friendship, "Charlie", "Immanuel")  # New relationship and new node

print("Friendships created or matched successfully.")


################# Using MATCH with MERGE for relationships  #################

# Function to create or match a FOLLOW relationship (directed)
def create_or_match_follower(tx, follower, followed):
    tx.run("""
    MATCH (a:User {name: $follower})
    MATCH (b:User {name: $followed})
    MERGE (a)-[:FOLLOW]->(b)
    """, follower=follower, followed=followed)

# Add some follower relationships to the database
with driver.session() as session:
    session.execute_write(create_or_match_follower, "Alice", "Charlie")  # Nodes and relationship already exist
    session.execute_write(create_or_match_follower, "Charlie", "Gabriella")  # Nodes already exist and new relationship
    session.execute_write(create_or_match_follower, "Jacob", "Bob")  # New relationship and new node

print("Follower relationships created or matched successfully.")


# Function to create or match a FRIEND relationship between two users
def create_or_match_friendship(tx, name1, name2):
    tx.run("""
    MATCH (a:User {name: $name1})
    MATCH (b:User {name: $name2})
    MERGE (a)-[:FRIEND]->(b)
    MERGE (b)-[:FRIEND]->(a)
    """, name1=name1, name2=name2)

# Add some friendships to the database
with driver.session() as session:
    session.execute_write(create_or_match_friendship, "Alice", "Charlie")  # Nodes and relationship already exist
    session.execute_write(create_or_match_friendship, "Gabriella", "Eli")  # Nodes already exist and new relationship
    session.execute_write(create_or_match_friendship, "Kevin", "Immanuel")  # New relationship and new nodes

print("Friendships created or matched successfully.")


Users created or matched successfully.
Follower relationships created or matched successfully.
Friendships created or matched successfully.
Follower relationships created or matched successfully.
Friendships created or matched successfully.


##### Explanation


###### Using `MERGE` to create or match nodes

- `MERGE (u:User {name: $name})`: Ensures that a node with the specified `name` exists. If a node with this `name` already exists, it will be matched. If not, a new `User` node will be created.  Unlike `CREATE`, which always creates a new node, `MERGE` will only create the node if it does not already exist. This prevents duplicate nodes.
- `SET u.age = $age`: Updates the `age` property of the node. If the node was newly created, it sets the `age` property. If the node already existed, it updates the property.

---

###### Using `MERGE` for both nodes and relationship

- `MERGE (a:User {name: $follower/$name1})` and `MERGE (b:User {name: $followed/$name2})`: Ensure that both users/nodes exist in the database. creating them if they don't already exist.
- `MERGE (a)-[:FOLLOW/FRIEND]->(b)`: Ensures that a specific relationship (FOLLOW/FRIEND) exists from `a` to `b`. If the relationship already exists, it is left unchanged; otherwise, it is created.

###### Using `MATCH` with `MERGE` for Relationships

- `MATCH (a:User {name: $follower/$name1})` and `MATCH (b:User {name: $followed/$name2})`: Find existing nodes in the database. If either node does not exist, this query will not create them.
- `MERGE (a)-[:FOLLOW/FRIEND]->(b)`: Ensures that a specific relationship (FOLLOW/FRIEND) exists from `a` and `b`. If the relationship does not exist, it is created; otherwise, it is left unchanged. It will be executed depending on the `MATCH` results.


**Comparison**:
  - **`MATCH` + `MERGE`** allows for more control and it is useful if we are sure the nodes already exist and we want to ensure the relationship is created or matched. `MATCH` is used to find existing nodes or relationships but does not create them if they do not exist.
  - **`MERGE` for both nodes and relationships** automatically handles the creation of nodes and relationships, which is simpler but may create nodes if they don't exist. `MERGE` is a combination of `MATCH` and `CREATE`. It will find the nodes and relationships if they exist or create them if they don't, making it more versatile for ensuring data integrity.
  
  
### Updating node properties

We may need to update specific properties of nodes. In this example, we'll focus on updating the age property.

In [10]:
# Function to update the age of a user node
def update_user_age(tx, name, new_age):
    tx.run("""
    MATCH (u:User {name: $name})
    SET u.age = $new_age
    """, name=name, new_age=new_age)

# Update the age for Hannah and Immanuel
with driver.session() as session:
    session.execute_write(update_user_age, "Hannah", 33)  # Setting new age for Hannah
    session.execute_write(update_user_age, "Immanuel", 27)  # Setting new age for Immanuel

print("User ages updated successfully.")

User ages updated successfully.


##### Explanation:

`SET` modifies the properties of nodes or relationships that were matched by the `MATCH` clause. If the property does not already exist, it will be created with the specified value. If the property already exists, it will be updated.

### Running queries
Now, let's run some Cypher queries to retrieve and analyze data.

In [11]:
# Function to get all users
def get_all_users(tx):
    result = tx.run("MATCH (u:User) RETURN u.name AS name, u.age AS age")
    return [(record["name"], record["age"]) for record in result]

# Retrieve and print all users
print("All users:")
with driver.session() as session:
    users = session.execute_write(get_all_users)
    for name, age in users:
        print(f"User: {name}, Age: {age}")
        
# Function to get all posts
def get_all_posts(tx):
    result = tx.run("MATCH (p:Post) RETURN p.id AS id, p.content AS content, p.timestamp AS timestamp")
    return [(record["id"], record["content"], record["timestamp"]) for record in result]

# Retrieve and print all posts
print("\nAll posts:")
with driver.session() as session:
    posts = session.execute_write(get_all_posts)
    for id, content, timestamp in posts:
        print(f"Post ID: {id}, Content: {content}, Timestamp: {timestamp}")
        
# Function to get all comments
def get_all_comments(tx):
    result = tx.run("MATCH (c:Comment) RETURN c.id AS id, c.content AS content, c.timestamp AS timestamp")
    return [(record["id"], record["content"], record["timestamp"]) for record in result]

# Retrieve and print all comments
print("\nAll comments:")
with driver.session() as session:
    comments = session.execute_write(get_all_comments)
    for id, content, timestamp in comments:
        print(f"Comment ID: {id}, Content: {content}, Timestamp: {timestamp}")

All users:
User: Gabriella, Age: 31
User: Hannah, Age: 33
User: Immanuel, Age: 27
User: Alice, Age: 30
User: Bob, Age: 25
User: Charlie, Age: 35
User: Eli, Age: 33
User: Felix, Age: 23

All posts:
Post ID: 1, Content: Hello, world!, Timestamp: 2024-07-16T10:00:00.000000000+00:00
Post ID: 2, Content: Good night!, Timestamp: 2024-07-16T11:00:00.000000000+00:00

All comments:
Comment ID: 2, Content: Very informative., Timestamp: 2024-07-16T12:30:00.000000000+00:00
Comment ID: 1, Content: Great post!, Timestamp: 2024-07-16T12:00:00.000000000+00:00


##### Explanation of the MATCH and RETURN query

The `MATCH` query is used to find nodes, and the `RETURN` query is used to get specific data from those nodes. Here's the structure:

```cypher
MATCH (alias:Label)
RETURN alias.property1, alias.property2, ...
```

- `MATCH (alias:Label)`: Finds nodes with the given label.
- `RETURN alias.property1, alias.property2, ...`: Returns the specified properties of the matched nodes.

In our example:
```cypher
MATCH (u:User)
RETURN u.name AS name, u.age AS age
```
- `MATCH (u:User)`: Finds all nodes with the label `User`. `u` is an alias for these nodes.
- `RETURN u.name AS name, u.age AS age`: Returns the `name` and `age` properties of the matched nodes.

In [12]:
# Function to get friends of a user
def get_friends(tx, name):
    result = tx.run("""
    MATCH (u:User {name: $name})-[:FRIEND]->(friend)
    RETURN friend.name AS name, friend.age AS age
    """, name=name)
    return [(record["name"], record["age"]) for record in result]

# Retrieve and print friends of Alice
with driver.session() as session:
    friends = session.execute_write(get_friends, "Alice")
    print("Alice's friends:")
    for name, age in friends:
        print(f"Friend: {name}, Age: {age}")

Alice's friends:
Friend: Charlie, Age: 35
Friend: Bob, Age: 25


##### Explanation of the MATCH and RETURN query with relationships
In this query, we are also matching relationships. Here's the structure:

```cypher
MATCH (alias1:Label {property1: value1})-[:RELATIONSHIP_TYPE]->(alias2)
RETURN alias2.property1, alias2.property2, ...
```

- `MATCH (alias1:Label {property1: value1})-[:RELATIONSHIP_TYPE]->(alias2)`: Finds nodes and their relationships.
- `RETURN alias2.property1, alias2.property2, ...`: Returns the specified properties of the related nodes.

In our example, we're finding users by their name and then finding their friends through the `FRIEND` relationship.:
```cypher
MATCH (u:User {name: $name})-[:FRIEND]->(friend)
RETURN friend.name AS name, friend.age AS age
```
- `MATCH (u:User {name: $name})-[:FRIEND]->(friend)`: Finds users named `$name` and their friends.
    - `MATCH (u:User {name: $name})`: Finds the user node with the specified name. `$name` is a parameter.
    - `[:FRIEND]->(friend)`: Finds nodes that have a `FRIEND` relationship from the matched user. `friend` is an alias for these related nodes.
- `RETURN friend.name AS name, friend.age AS age`: Returns the `name` and `age` properties of the friends.

In [13]:
# Function to get friends of friends
def get_friends_of_friends(tx, name):
    result = tx.run("""
    MATCH (u:User {name: $name})-[:FRIEND]->(:User)-[:FRIEND]->(fof)
    RETURN fof.name AS name, fof.age AS age
    """, name=name)
    return [(record["name"], record["age"]) for record in result]

# Retrieve and print friends of friends of Alice
with driver.session() as session:
    friends_of_friends = session.execute_write(get_friends_of_friends, "Alice")
    print("Alice's friends of friends:")
    for name, age in friends_of_friends:
        print(f"Friend of Friend: {name}, Age: {age}")

Alice's friends of friends:
Friend of Friend: Immanuel, Age: 27
Friend of Friend: Alice, Age: 30
Friend of Friend: Felix, Age: 23
Friend of Friend: Alice, Age: 30


##### Explanation of the MATCH and RETURN query for friends of friends

This query finds nodes that are connected through two relationships. Here's the structure:

```cypher
MATCH (alias1:Label {property1: value1})-[:RELATIONSHIP_TYPE1]->(:Label)-[:RELATIONSHIP_TYPE2]->(alias2)
RETURN alias2.property1, alias2.property2, ...
```

- `MATCH (alias1:Label {property1: value1})-[:RELATIONSHIP_TYPE1]->(:Label)-[:RELATIONSHIP_TYPE2]->(alias2)`: Finds nodes connected through two relationships.
- `RETURN alias2.property1, alias2.property2, ...`: Returns the specified properties of the nodes at the end of the relationships.

In our example:
```cypher
MATCH (u:User {name: $name})-[:FRIEND]->(:User)-[:FRIEND]->(fof)
RETURN fof.name AS name, fof.age AS age
```
- `MATCH (u:User {name: $name})-[:FRIEND]->(:User)-[:FRIEND]->(fof)`: Finds users named `$name` and their friends' friends.
    - `MATCH (u:User {name: $name})`: Finds the user node with the specified name.
    - `[:FRIEND]->(:User)`: Finds user nodes that are friends of the matched user. The friend nodes are not given an alias here.
    - `[:FRIEND]->(fof)`: Finds nodes that are friends of the friend nodes. `fof` is an alias for these friends of friends.
- `RETURN fof.name AS name, fof.age AS age`: Returns the `name` and `age` properties of the friends' friends.

In [14]:
# Function to get the count of different types of nodes
def count_nodes(tx):
    result = tx.run("""
    MATCH (n)
    RETURN labels(n)[0] AS nodeType, count(*) AS count
    """)
    return [(record["nodeType"], record["count"]) for record in result]

# Retrieve and print the count of different types of nodes
with driver.session() as session:
    counts = session.execute_write(count_nodes)
    print("Node counts:")
    for nodeType, count in counts:
        print(f"Node Type: {nodeType}, Count: {count}")

# Function to get the count of different types of relationships
def count_relationships(tx):
    result = tx.run("""
    MATCH ()-[r]->()
    RETURN type(r) AS relationshipType, count(*) AS count
    """)
    return [(record["relationshipType"], record["count"]) for record in result]

# Retrieve and print the count of different types of relationships
with driver.session() as session:
    counts = session.execute_write(count_relationships)
    print("\nRelationship counts:")
    for relationshipType, count in counts:
        print(f"Relationship Type: {relationshipType}, Count: {count}")

Node counts:
Node Type: User, Count: 8
Node Type: Comment, Count: 2
Node Type: Post, Count: 2

Relationship counts:
Relationship Type: HAS_COMMENT, Count: 2
Relationship Type: COMMENTED, Count: 2
Relationship Type: LIKES, Count: 2
Relationship Type: FOLLOW, Count: 5
Relationship Type: FRIEND, Count: 10
Relationship Type: POSTED, Count: 2


##### Explanation

* **`MATCH` Clause**:
    - In the context of counting nodes or relationships, `MATCH (n)` or `MATCH ()-[r]->()` specifies that we are looking at all nodes or relationships respectively.

* **`labels(n)` Function**:
    - `labels(n)` returns a list of labels for a node `n`. In the example:
    ```cypher
    RETURN labels(n)[0] AS nodeType
    ```

    - This line retrieves the first label of the node `n` and assigns it the alias `nodeType`.

* **`type(r)` Function**:
    - `type(r)` returns the type of a relationship `r`. In the example:
    ```cypher
    RETURN type(r) AS relationshipType
    ```

    - This line retrieves the type of the relationship and assigns it the alias `relationshipType`.

* **`count(*)` Function**: Counts the number of elements in the result set. It is used to aggregate the number of nodes or relationships.

In [15]:
# Function to get users who liked a specific post
def get_users_who_liked_post(tx, post_id):
    result = tx.run("""
    MATCH (u:User)-[r:LIKES]->(p:Post {id: $post_id})
    RETURN u.name AS name, r.timestamp AS liked_at
    """, post_id=post_id)
    return [(record["name"], record["liked_at"]) for record in result]

# Retrieve and print users who liked post with ID 1
with driver.session() as session:
    likers = session.execute_write(get_users_who_liked_post, 1)
    print("Users who liked post ID 1:")
    for name, liked_at in likers:
        print(f"User: {name}, Liked at: {liked_at}")


Users who liked post ID 1:
User: Alice, Liked at: 2024-07-16T12:00:00Z


##### Explanation

* `MATCH (u:User)-[r:LIKES]->(p:Post {id: $post_id})`: Finds users who have a LIKES relationship to the post with the specified id.
* `RETURN u.name AS name, r.timestamp AS liked_at`: Returns the name property of the users and the timestamp property of the LIKES relationship.

In [16]:
# Function to get posts made by a specific user
def get_posts_by_user(tx, user_name):
    result = tx.run("""
    MATCH (u:User {name: $user_name})-[:POSTED]->(p:Post)
    RETURN p.id AS id, p.content AS content, p.timestamp AS timestamp
    """, user_name=user_name)
    return [(record["id"], record["content"], record["timestamp"]) for record in result]

# Retrieve and print posts made by Alice
with driver.session() as session:
    posts = session.execute_write(get_posts_by_user, "Alice")
    print("Posts made by Alice:")
    for id, content, timestamp in posts:
        print(f"Post ID: {id}, Content: {content}, Timestamp: {timestamp}")


# Function to get comments made by a specific user
def get_comments_by_user(tx, user_name):
    result = tx.run("""
    MATCH (u:User {name: $user_name})-[:COMMENTED]->(c:Comment)
    RETURN c.id AS id, c.content AS content, c.timestamp AS timestamp
    """, user_name=user_name)
    return [(record["id"], record["content"], record["timestamp"]) for record in result]

# Retrieve and print comments made by Alice
with driver.session() as session:
    comments = session.execute_write(get_comments_by_user, "Alice")
    print("\nComments made by Alice:")
    for id, content, timestamp in comments:
        print(f"Comment ID: {id}, Content: {content}, Timestamp: {timestamp}")
        

# Function to get comments on a specific post
def get_comments_on_post(tx, post_id):
    result = tx.run("""
    MATCH (p:Post {id: $post_id})-[:HAS_COMMENT]->(c:Comment)
    RETURN c.id AS id, c.content AS content, c.timestamp AS timestamp
    """, post_id=post_id)
    return [(record["id"], record["content"], record["timestamp"]) for record in result]

# Retrieve and print comments on post with ID 1
with driver.session() as session:
    comments = session.execute_write(get_comments_on_post, 1)
    print("\nComments on Post ID 1:")
    for id, content, timestamp in comments:
        print(f"Comment ID: {id}, Content: {content}, Timestamp: {timestamp}")

Posts made by Alice:
Post ID: 1, Content: Hello, world!, Timestamp: 2024-07-16T10:00:00.000000000+00:00

Comments made by Alice:
Comment ID: 1, Content: Great post!, Timestamp: 2024-07-16T12:00:00.000000000+00:00

Comments on Post ID 1:
Comment ID: 2, Content: Very informative., Timestamp: 2024-07-16T12:30:00.000000000+00:00


##### Explanation

* `MATCH (u:User {name: $user_name})-[:POSTED]->(p:Post)`: Finds posts that a user with the specified name has posted.
* `RETURN p.id AS id, p.content AS content, p.timestamp AS timestamp`: Returns the id, content, and timestamp properties of the posts.

In [17]:
# Function to recommend friends to a user with pagination
def recommend_friends_with_pagination(tx, user_name, skip, limit):
    result = tx.run("""
    MATCH (u:User {name: $user_name})-[:FRIEND]->(friend)-[:FRIEND]->(fof)
    WHERE NOT (u)-[:FRIEND]->(fof) AND u <> fof
    RETURN fof.name AS name, count(friend) AS mutualFriends
    ORDER BY mutualFriends DESC
    SKIP $skip
    LIMIT $limit
    """, user_name=user_name, skip=skip, limit=limit)
    return [(record["name"], record["mutualFriends"]) for record in result]

# Function to recommend friends for a list of users with pagination
def recommend_friends_for_users_with_pagination(driver, user_names, skip, limit):
    for user_name in user_names:
        with driver.session() as session:
            recommendations = session.execute_write(recommend_friends_with_pagination, user_name, skip, limit)
            print(f"\nFriend recommendations for {user_name}:")
            for name, mutualFriends in recommendations:
                print(f"Recommended Friend: {name}, Mutual Friends: {mutualFriends}")

# Retrieve and print friend recommendations with pagination for a list of users
user_names = ["Alice", "Bob", "Charlie"]
recommend_friends_for_users_with_pagination(driver, user_names, skip=0, limit=5)



Friend recommendations for Alice:
Recommended Friend: Immanuel, Mutual Friends: 1
Recommended Friend: Felix, Mutual Friends: 1

Friend recommendations for Bob:
Recommended Friend: Charlie, Mutual Friends: 1

Friend recommendations for Charlie:
Recommended Friend: Bob, Mutual Friends: 1


##### Explanation

* **`WHERE` Clause**:
    - The `WHERE` clause is used to filter the results of the `MATCH` clause based on specific conditions. 
    ```cypher
    WHERE NOT (u)-[:FRIEND]->(fof) AND u <> fof
    ```

    - This line ensures that `fof` (friend of friend) is not already a friend of the user `u` and that `fof` is not the user `u` itself.

* **`ORDER BY` Clause**:
    - The `ORDER BY` clause is used to sort the results based on one or more expressions. 
    ```cypher
    ORDER BY mutualFriends DESC
    ```

    - This line sorts the results by the number of mutual friends in descending order.
    
* **`SKIP` Clause**:
    - The `SKIP` clause is used to skip a specified number of results before starting to return results. 
    ```cypher
    SKIP $skip
    ```

    - This line skips a specified number of results (e.g., 0 in this case).

* **`LIMIT` Clause**:
    - The `LIMIT` clause is used to specify the maximum number of results to return. 
    ```cypher
    LIMIT $limit
    ```

    - This line limits the results to the top 5 recommendations.

* **`NOT` Keyword**:
    - The `NOT` keyword is used to exclude results that match a certain condition. 
    ```cypher
    WHERE NOT (u)-[:FRIEND]->(fof)
    ```

    - This line excludes `fof` from the results if there is already a `FRIEND` relationship between `u` and `fof`.

* **`<>` Operator**:
    - The `<>` operator is used to denote inequality.
    ```cypher
    AND u <> fof
    ```

    - This line ensures that `fof` is not the same as `u` (the user for whom we are recommending friends).


In [18]:
# Function to find the shortest path between two users
def get_shortest_path(tx, user1, user2):
    result = tx.run("""
    MATCH p = shortestPath((u1:User {name: $user1})-[:FRIEND*]-(u2:User {name: $user2}))
    RETURN p
    """, user1=user1, user2=user2)
    return result.single()

# Retrieve and print the shortest path between Alice and Charlie
with driver.session() as session:
    path = session.execute_write(get_shortest_path, "Alice", "Charlie")
    print("Shortest path between Alice and Charlie:")
    nodes = [node["name"] for node in path["p"].nodes]
    print(" -> ".join(nodes))
    
    print("\nShortest path between Felix and Charlie:")
    path = session.execute_write(get_shortest_path, "Felix", "Immanuel")
    nodes = [node["name"] for node in path["p"].nodes]
    print(" -> ".join(nodes))

Shortest path between Alice and Charlie:
Alice -> Charlie

Shortest path between Felix and Charlie:
Felix -> Bob -> Alice -> Charlie -> Immanuel


##### Explanation

* **`shortestPath` function** is used to find the shortest path between two nodes. 
    
    ```
    shortestPath((u1:User {name: $user1})-[:FRIEND*]-(u2:User {name: $user2}))
    ```

    - This function finds the shortest path between `u1` and `u2` traversing `FRIEND` relationships.
    
* **`*` in `[:FRIEND*]`** indicates that the relationship can traverse multiple hops or steps. `[:FRIEND*]` means that the query should consider paths that include zero or more `FRIEND` relationships. The `*` specifies that the path can be of any length, including direct connections or longer paths through other nodes.

  - For example:
    - `[:FRIEND]` would only match direct connections between nodes.
    - `[:FRIEND*]` allows the path to include nodes that are connected through one or more `FRIEND` relationships.

In [19]:
# Function to find all patterns of user -> friend -> user -> friend -> user -> follow -> user
def find_user_patterns(tx):
    result = tx.run("""
    MATCH (a:User)-[:FRIEND]->(b:User)-[:FRIEND]->(c:User)-[:FOLLOW]->(d:User)
    WHERE a <> b AND a <> c AND a <> d AND b <> c AND b <> d AND c <> d
    RETURN a.name AS startUser, b.name AS firstFriend, c.name AS secondFriend, d.name AS followedUser
    """)
    return [(record["startUser"], record["firstFriend"], record["secondFriend"], record["followedUser"]) for record in result]

# Execute the function and print results
with driver.session() as session:
    patterns = session.execute_read(find_user_patterns)
    print("Patterns found:")
    for startUser, firstFriend, secondFriend, followedUser in patterns:
        print(f"{startUser} -> {firstFriend} -> {secondFriend} -> {followedUser}")
        
        
# Function to find all patterns of user -> friend -> user <- follow <- user -> friend -> user
def find_user_patterns(tx):
    result = tx.run("""
    MATCH (a:User)-[:FRIEND]->(b:User)<-[:FOLLOW]-(c:User)-[:FRIEND]->(d:User)
    WHERE a <> b AND a <> c AND a <> d AND b <> c AND b <> d AND c <> d
    RETURN a.name AS startUser, b.name AS firstFriend, c.name AS FriendFollowerUser, d.name AS followerFriendUser
    """)
    return [(record["startUser"], record["firstFriend"], record["FriendFollowerUser"], record["followerFriendUser"]) for record in result]

# Execute the function and print results
with driver.session() as session:
    patterns = session.execute_read(find_user_patterns)
    print("\nPatterns found:")
    for startUser, firstFriend, FriendFollowerUser, followerFriendUser in patterns:
        print(f"{startUser} -> {firstFriend} <- {FriendFollowerUser} -> {followerFriendUser}")
        
        
# Function to find users connected through an intermediate node
def find_connections_through_intermediate(tx):
    result = tx.run("""
    MATCH (a:User)-->(x)<--(b:User)
    WHERE a <> b
    RETURN DISTINCT a.name AS startUser, x.name AS intermediateNode, b.name AS endUser
    """)
    return [(record["startUser"], record["intermediateNode"], record["endUser"]) for record in result]

# Execute the function and print results
with driver.session() as session:
    connections = session.execute_read(find_connections_through_intermediate)
    print("\nConnections through an intermediate node found:")
    for startUser, intermediateNode, endUser in connections:
        print(f"{startUser} -- {intermediateNode} -- {endUser}")

Patterns found:
Felix -> Bob -> Alice -> Charlie
Charlie -> Alice -> Bob -> Eli
Bob -> Alice -> Charlie -> Gabriella

Patterns found:
Immanuel -> Charlie <- Alice -> Bob
Gabriella -> Eli <- Bob -> Felix
Gabriella -> Eli <- Bob -> Alice
Felix -> Bob <- Eli -> Gabriella
Alice -> Bob <- Eli -> Gabriella
Eli -> Gabriella <- Charlie -> Immanuel
Eli -> Gabriella <- Charlie -> Alice

Connections through an intermediate node found:
Bob -- Eli -- Gabriella
Bob -- Felix -- Hannah
Alice -- Charlie -- Immanuel
Immanuel -- Charlie -- Alice
Felix -- Bob -- Alice
Eli -- Bob -- Alice
Hannah -- Felix -- Bob
Charlie -- Alice -- Bob
Gabriella -- Eli -- Bob
Eli -- Gabriella -- Charlie
Bob -- Alice -- Charlie
Charlie -- Gabriella -- Eli
Felix -- Bob -- Eli
Alice -- Bob -- Eli
Alice -- Bob -- Felix
Eli -- Bob -- Felix


#### Finding directed relationships in Cypher queries

The direction of the relationship is indicated by `->` (outgoing) or `<-` (incoming).
  - Example: **`(a:User)-[:FRIEND]->(b:User)`** means user `a` is friends with user `b`.
  - Example: **`(b:User)<-[:FOLLOW]-(c:User)`** means user `c` follows user `b`.

#### Finding both directions of a relationship

To find relationships in both directions, we can use the `-[]-` syntax, which matches relationships regardless of direction.
- Example: **`(a:User)-[:FRIEND]-(b:User)`** means there is a `FRIEND` relationship between user `a` and user `b`, regardless of who initiated the friendship.
- Example: **`(b:User)-[:FOLLOW]-(c:User)`** means there is a `FOLLOW` relationship between user `b` and user `c`, regardless of who follows whom.


#### Finding connections through an intermediate node

To find patterns where two nodes are connected through a third, intermediate node, we can use the `(a)-->()<--(b)` pattern, which identifies relationships between nodes that share a common intermediary.
- Example: **`(a:User)-->(x)<--(b:User)`** finds all users `a` and `b` who are connected through an intermediate user `x`. `(a:User)-->(x)` means that user `a` has a direct relationship (in any direction) with an intermediate node `x`. `(x)<--(b:User)` means that user `b` has a direct relationship (in any direction) with the same intermediate node `x`.

In [20]:
# Function to find users with more than one friend
def find_users_with_more_than_one_friend(tx):
    result = tx.run("""
    MATCH (u:User)-[:FRIEND]->(friend)
    WITH u, COUNT(friend) AS friendCount
    WHERE friendCount > 1
    RETURN u.name AS userName, friendCount
    """)
    return [(record["userName"], record["friendCount"]) for record in result]

# Execute the function and print results
with driver.session() as session:
    users = session.execute_read(find_users_with_more_than_one_friend)
    print("Users with more than one friend:")
    for userName, friendCount in users:
        print(f"{userName} has {friendCount} friends")

        
# Function to create a mentor relationship for friends older than 30
def create_mentor_relationship(tx):
    tx.run("""
    MATCH (a:User {name: 'Alice'})-[:FRIEND]->(b:User)
    WITH a, b
    WHERE b.age > 30
    CREATE (a)-[:MENTOR]->(b)
    RETURN a.name AS mentor, b.name AS mentee
    """)

# Execute the function
with driver.session() as session:
    session.execute_write(create_mentor_relationship)

print("\nMentor relationships created successfully.")

Users with more than one friend:
Alice has 2 friends
Bob has 2 friends
Charlie has 2 friends

Mentor relationships created successfully.


##### Using the `WITH` clause
The `WITH` clause is used to chain multiple query parts together, allowing us to pass the results of one part of the query to the next. This is useful for performing intermediate calculations, filtering results, or aggregating data before further processing.

1. Example: Finding users with more than one friend:
    ```cypher
    MATCH (u:User)-[:FRIEND]->(friend)
    WITH u, COUNT(friend) AS friendCount
    WHERE friendCount > 1
    RETURN u.name, friendCount
    ```
    - **MATCH**: Finds all users and their friends.
    - **WITH**: Passes each user and the count of their friends to the next part of the query.
    - **WHERE**: Filters users who have more than one friend.
    - **RETURN**: Returns the user's name and the count of their friends.

2. Example: Creating a relationship with intermediate filtering:
    ```cypher
    MATCH (a:User {name: 'Alice'})-[:FRIEND]->(b:User)
    WITH a, b
    WHERE b.age > 30
    CREATE (a)-[:MENTOR]->(b)
    RETURN a.name, b.name
    ```
    - **MATCH**: Finds Alice and her friends.
    - **WITH**: Passes Alice and her friends to the next part of the query.
    - **WHERE**: Filters friends who are older than 30.
    - **CREATE**: Creates a `MENTOR` relationship between Alice and those friends.
    - **RETURN**: Returns the names of Alice and her mentees.

The `WITH` clause allows breaking complex queries into manageable parts. We can aggregate data and pass the results to the next part of the query. It also helps in scoping variables to avoid naming conflicts and allows reusing variables in subsequent query parts.

In [21]:
# Function to collect friends of each user
def collect_friends(tx):
    result = tx.run("""
    MATCH (u:User)-[:FRIEND]->(friend)
    WITH u, COLLECT(friend.name) AS friends
    RETURN u.name AS userName, friends
    """)
    return [(record["userName"], record["friends"]) for record in result]

# Execute the function and print results
with driver.session() as session:
    users = session.execute_read(collect_friends)
    print("Users and their friends:")
    for userName, friends in users:
        print(f"{userName} is friends with {', '.join(friends)}")

        
# Function to collect and count follow relationships
def collect_and_count_follows(tx):
    result = tx.run("""
    MATCH (u:User)-[:FOLLOW]->(followed)
    WITH u, COLLECT(followed.name) AS follows, COUNT(followed) AS followCount
    RETURN u.name AS userName, follows, followCount
    """)
    return [(record["userName"], record["follows"], record["followCount"]) for record in result]

# Execute the function and print results
with driver.session() as session:
    users = session.execute_read(collect_and_count_follows)
    print("\nUsers and their follows:")
    for userName, follows, followCount in users:
        print(f"{userName} follows {followCount} users: {', '.join(follows)}")


Users and their friends:
Gabriella is friends with Eli
Immanuel is friends with Charlie
Alice is friends with Charlie, Bob
Bob is friends with Felix, Alice
Charlie is friends with Immanuel, Alice
Eli is friends with Gabriella
Felix is friends with Bob

Users and their follows:
Alice follows 1 users: Charlie
Bob follows 1 users: Eli
Eli follows 1 users: Bob
Hannah follows 1 users: Felix
Charlie follows 1 users: Gabriella


##### Using the `COLLECT` function
The `COLLECT` function is used to aggregate values into a list. This is particularly useful when we need to group related values together.

1. Example: collecting friends of each user:
    ```cypher
    MATCH (u:User)-[:FRIEND]->(friend)
    WITH u, COLLECT(friend.name) AS friends
    RETURN u.name, friends
    ```
    - **MATCH**: Finds all users and their friends.
    - **WITH**: Groups friends of each user into a list.
    - **RETURN**: Returns the user's name and a list of their friends' names.

2. Example: Collecting and counting follow relationships:
    ```cypher
    MATCH (u:User)-[:FOLLOW]->(followed)
    WITH u, COLLECT(followed.name) AS follows, COUNT(followed) AS followCount
    RETURN u.name, follows, followCount
    ```
    - **MATCH**: Finds all users and the users they follow.
    - **WITH**: Groups the followed users' names into a list and counts them.
    - **RETURN**: Returns the user's name, the list of followed users, and the count of followed users.

In [22]:
# Function to find users who are either followed by 'Alice' or are friends with 'Bob'
def find_users_followed_by_alice_or_friends_with_bob(tx):
    result = tx.run("""
    MATCH (alice:User {name: 'Alice'})-[:FOLLOW]->(u:User)
    RETURN u.name AS name, 'followed_by_Alice' AS relationship_type
    UNION
    MATCH (bob:User {name: 'Bob'})-[:FRIEND]->(u:User)
    RETURN u.name AS name, 'friend_of_Bob' AS relationship_type
    """)
    return [(record["name"], record["relationship_type"]) for record in result]

# Execute the function and print results
with driver.session() as session:
    users = session.execute_read(find_users_followed_by_alice_or_friends_with_bob)
    print("Users followed by Alice or friends with Bob:")
    for name, relationship_type in users:
        print(f"{name} ({relationship_type})")

Users followed by Alice or friends with Bob:
Charlie (followed_by_Alice)
Felix (friend_of_Bob)
Alice (friend_of_Bob)


##### Using the `UNION` clause
The `UNION` clause is used to combine the results of two or more queries into a single result set. It removes duplicate rows from the result set by default. If we want to include duplicates, we can use `UNION ALL`. In the example, `UNION` combines the results of two `MATCH` queries:
1. The first `MATCH` query finds users who are followed by 'Alice' and returns their names along with the relationship type 'followed_by_Alice'.
2. The second `MATCH` query finds users who are friends with 'Bob' and returns their names along with the relationship type 'friend_of_Bob'.

In [23]:
# Function to add multiple friends for a user
def add_friends(tx, user_name, friends):
    tx.run("""
    MATCH (u:User {name: $user_name})
    FOREACH (friend IN $friends |
      MERGE (f:User {name: friend})
      MERGE (u)-[:FRIEND]->(f)
    )
    """, user_name=user_name, friends=friends)

# Execute the function
with driver.session() as session:
    session.execute_write(add_friends, "Alice", ["David", "Eva", "Frank"])

print("Friends added successfully.")



# Function to create multiple relationships from a list of friends
def create_friendships(tx, user_name, friends_list):
    tx.run("""
    MATCH (u:User {name: $user_name})
    UNWIND $friends_list AS friendName
    MERGE (f:User {name: friendName})
    MERGE (u)-[:FRIEND]->(f)
    """, user_name=user_name, friends_list=friends_list)

# Execute the function
with driver.session() as session:
    session.execute_write(create_friendships, "Alice", ["Tom", "Bob", "Esther", "Daniella"])

print("Friendships created successfully.")

Friends added successfully.
Friendships created successfully.


##### Using the `FOREACH` clause
The `FOREACH` clause is used to perform updating operations on each element of a collection. It is useful for iterating over a list of elements and performing a set of actions on each element. In the example, `FOREACH` iterates over each element in the `friends` list. For each friend, it either finds or creates a `User` node and creates a `FRIEND` relationship from the user `Alice` to the friend.

##### Using the `UNWIND` Clause
`UNWIND` is used to expand a list into a set of rows. It transforms a single list of values into individual rows, which allows us to process each item in the list separately. This is particularly useful when we have a list of items and we need to perform operations on each item.

**Comparison**:

`UNWIND` and `FOREACH` are both used to handle lists in Cypher queries.
* `UNWIND`: Transforms a list into separate rows for more general processing, often followed by other Cypher operations. It provides more flexibility and can be combined with other Cypher clauses (e.g., `WITH`) for more complex queries and aggregations.
* `FOREACH`: Executes commands for each item in a list without transforming it into separate rows. It is more straightforward way for executing actions on each list item without needing to manage intermediate results.

In [24]:
# Function to categorize users by age group
def categorize_users_by_age(tx):
    result = tx.run("""
    MATCH (u:User)
    RETURN u.name AS name,
           CASE 
             WHEN u.age < 20 THEN 'teenager'
             WHEN u.age < 30 THEN 'young adult'
             ELSE 'adult'
           END AS ageGroup
    """)
    return [(record["name"], record["ageGroup"]) for record in result]

# Execute the function and print results
with driver.session() as session:
    users = session.execute_read(categorize_users_by_age)
    print("Users categorized by age group:")
    for name, ageGroup in users:
        print(f"{name}: {ageGroup}")

Users categorized by age group:
Gabriella: adult
Hannah: adult
Immanuel: young adult
David: adult
Eva: adult
Frank: adult
Tom: adult
Esther: adult
Daniella: adult
Alice: adult
Bob: young adult
Charlie: adult
Eli: adult
Felix: young adult


##### Using the `CASE` clause
The `CASE` clause is used to implement conditional logic within Cypher queries. It allows you to return different values based on specified conditions, similar to an `IF-THEN-ELSE` statement in other programming languages. In our example, `CASE` evaluates the `u.age` property for each user and returns a corresponding age group. The `WHEN` clauses define the conditions for different age groups: 'teenager', 'young adult', and 'adult'. The result set contains the user names and their respective age groups.

In [25]:
# Function to find users and their optional followers
def find_users_with_optional_followers(tx):
    result = tx.run("""
    MATCH (u:User)
    OPTIONAL MATCH (u)-[:FOLLOW]->(f:User)
    RETURN u.name AS userName, COLLECT(f.name) AS followers
    """)
    return [(record["userName"], record["followers"]) for record in result]

# Execute the function and print results
with driver.session() as session:
    user_followers = session.execute_read(find_users_with_optional_followers)
    print("Users and their optional followers:")
    for userName, followers in user_followers:
        followers_list = ", ".join(followers) if followers else "No followers"
        print(f"{userName}: {followers_list}")

Users and their optional followers:
Gabriella: No followers
Hannah: Felix
Immanuel: No followers
David: No followers
Eva: No followers
Frank: No followers
Tom: No followers
Esther: No followers
Daniella: No followers
Alice: Charlie
Bob: Eli
Charlie: Gabriella
Eli: Bob
Felix: No followers


##### Using the `OPTIONAL MATCH` clause
`OPTIONAL MATCH` is used to include patterns that might not exist in the graph. Unlike `MATCH`, which requires a pattern to be present for the query to return results, `OPTIONAL MATCH` returns results even if the pattern is missing. This is useful for including optional data in the results without filtering out nodes or relationships that might not have that data.

In [26]:
# Find posts within a certain time range
def find_posts_in_time_range(tx, start_time, end_time):
    result = tx.run("""
    MATCH (p:Post)
    WHERE p.timestamp >= datetime($start_time) AND p.timestamp <= datetime($end_time)
    RETURN p.id AS postId, p.content AS content, p.timestamp AS timestamp
    """, start_time=start_time, end_time=end_time)
    return [(record["postId"], record["content"], record["timestamp"]) for record in result]

# Execute the function and print results
with driver.session() as session:
    posts = session.execute_read(find_posts_in_time_range, "2024-07-16T00:00:00Z", "2024-07-16T23:59:59Z")
    print("Posts within the specified time range:")
    for postId, content, timestamp in posts:
        print(f"Post ID: {postId}, Content: {content}, Timestamp: {timestamp}")


#Find posts within a certain distance from a given point
def find_posts_near_location(tx, latitude, longitude, max_distance):
    result = tx.run("""
    MATCH (p:Post)
    WHERE point.distance(p.location, point({latitude: $latitude, longitude: $longitude})) < $max_distance
    RETURN p.id AS postId, p.content AS content, p.location AS location
    """, latitude=latitude, longitude=longitude, max_distance=max_distance)
    return [(record["postId"], record["content"], record["location"]) for record in result]

# Execute the function and print results
with driver.session() as session:
    posts = session.execute_read(find_posts_near_location, 40.7128, -74.0060, 10000)  # 10 km radius
    print("\nPosts within the specified distance from the location:")
    for postId, content, location in posts:
        print(f"Post ID: {postId}, Content: {content}, Location: {location}")


Posts within the specified time range:
Post ID: 1, Content: Hello, world!, Timestamp: 2024-07-16T10:00:00.000000000+00:00
Post ID: 2, Content: Good night!, Timestamp: 2024-07-16T11:00:00.000000000+00:00

Posts within the specified distance from the location:
Post ID: 1, Content: Hello, world!, Location: POINT(-74.006 40.7128)


### Deleting all nodes and relationships
To completely empty the database, we can delete all nodes and relationships. This is useful when we want to reset the database.

In [27]:
# Function to delete all nodes and relationships
def delete_all(tx):
    tx.run("MATCH (n) DETACH DELETE n")

# Delete all nodes and relationships in the database
with driver.session() as session:
    session.execute_write(delete_all)

print("All nodes and relationships deleted successfully.")

All nodes and relationships deleted successfully.


#### Closing the connection
Finally, close the connection to the database.

In [28]:
# Close the driver connection
driver.close()
print("Connection closed.")

Connection closed.
